In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import gc  # Import garbage collector module

from tensorflow.keras.applications import EfficientNetB3  # Import EfficientNetB3
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import Sequence

In [3]:
def calculate_difference_image(image1, image2):
    return cv2.absdiff(image1, image2)

In [4]:

class ImageDataGenerator(Sequence):
    def __init__(self, images, labels, batch_size):
        self.images = images
        self.labels = labels
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.images) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.images[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        return np.array(batch_x), np.array(batch_y)

In [5]:
def load_images_and_difference(authentic_dir, tampered_dir):
    images = []
    labels = []

    print("Loading authentic images...")
    for filename in os.listdir(authentic_dir):
        if filename.lower().endswith(('.jpg', '.bmp')):
            filepath = os.path.join(authentic_dir, filename)
            img = cv2.imread(filepath, cv2.IMREAD_COLOR)
            if img is not None:
                img_resized = cv2.resize(img, (224, 224)).astype(np.float32) / 255.0
                images.append(img_resized)
                labels.append(0)

    print("Loading tampered images and calculating differences...")
    for filename in os.listdir(tampered_dir):
        if filename.lower().endswith(('.jpg', '.tif')):
            tampered_img = cv2.imread(os.path.join(tampered_dir, filename), cv2.IMREAD_COLOR)
            tampered_img_resized = cv2.resize(tampered_img, (224, 224)).astype(np.float32) / 255.0
            
            match_found = False
            parts = filename.split("_")
            if len(parts) >= 7:
                source_id = parts[4][:3] + "_" + parts[4][3:]
                target_id = parts[5][:3] + "_" + parts[5][3:]
                for id_ in [source_id, target_id]:
                    for ext in ['.jpg', '.bmp']:
                        auth_filename = f"Au_{id_}{ext}"
                        auth_filepath = os.path.join(authentic_dir, auth_filename)
                        if os.path.exists(auth_filepath):
                            auth_img = cv2.imread(auth_filepath, cv2.IMREAD_COLOR)
                            auth_img_resized = cv2.resize(auth_img, (224, 224)).astype(np.float32) / 255.0
                            difference_img = calculate_difference_image(tampered_img_resized, auth_img_resized)
                            images.append(difference_img)
                            labels.append(1)
                            match_found = True
                            break
            if not match_found:
                print(f"No authentic match found for {filename}, using tampered image directly.")
                images.append(tampered_img_resized)
                labels.append(1)

    return np.array(images), np.array(labels)

In [5]:
authentic_dir = 'D:\SASTRA Files\Sem-6\MINI PROJECT\Digital Image Forgery Detection\CASIA2 Dataset\Au'
tampered_dir = 'D:\SASTRA Files\Sem-6\MINI PROJECT\Digital Image Forgery Detection\CASIA2 Dataset\Tp'

images, labels = load_images_and_difference(authentic_dir, tampered_dir)

Loading authentic images...
Loading tampered images and calculating differences...
No authentic match found for Tp_S_CNN_S_N_cha0003_cha00003_00323.tif, using tampered image directly.
No authentic match found for Tp_S_NNN_M_N_pla0006_pla00006_01128.tif, using tampered image directly.


In [6]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
print(f"Train set size: {X_train.shape[0]}, Test set size: {X_test.shape[0]}")

Train set size: 10091, Test set size: 2523


In [7]:
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))  # Use EfficientNetB0 here



43941136/43941136 [==============================] - 121s 3us/step


In [8]:
for layer in base_model.layers:
    layer.trainable = False

In [9]:
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

In [10]:
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
class TrainingMonitor(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"Starting epoch {epoch+1}")

    def on_epoch_end(self, epoch, logs=None):
        print(f"Finished epoch {epoch+1}")
        print(f"Loss: {logs['loss']}, Accuracy: {logs['accuracy']}, Val Loss: {logs['val_loss']}, Val Accuracy: {logs['val_accuracy']}")

In [12]:
batch_size = 16
train_generator = ImageDataGenerator(X_train, y_train, batch_size)
test_generator = ImageDataGenerator(X_test, y_test, batch_size)

In [13]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
    callbacks=[TrainingMonitor()],
    verbose=2)

Starting epoch 1
Epoch 1/10



Finished epoch 1
Loss: 0.42683446407318115, Accuracy: 0.8130016922950745, Val Loss: 0.30516916513442993, Val Accuracy: 0.8965517282485962
631/631 - 560s - loss: 0.4268 - accuracy: 0.8130 - val_loss: 0.3052 - val_accuracy: 0.8966 - 560s/epoch - 888ms/step
Starting epoch 2
Epoch 2/10
Finished epoch 2
Loss: 0.39181169867515564, Accuracy: 0.8451095223426819, Val Loss: 0.33440345525741577, Val Accuracy: 0.8581054210662842
631/631 - 477s - loss: 0.3918 - accuracy: 0.8451 - val_loss: 0.3344 - val_accuracy: 0.8581 - 477s/epoch - 757ms/step
Starting epoch 3
Epoch 3/10
Finished epoch 3
Loss: 0.3911387622356415, Accuracy: 0.850361704826355, Val Loss: 0.322979211807251, Val Accuracy: 0.8699960112571716
631/631 - 435s - loss: 0.3911 - accuracy: 0.8504 - val_loss: 0.3230 - val_accuracy: 0.8700 - 435s/epoch - 689ms/step
Starting epoch 4
Epoch 4/10
Finished epoch 4
Loss: 0.3909948468208313, Accuracy: 0.8530373573303223, Val Loss: 0.297611802816391, Val Accuracy: 0.9056678414344788
631/631 - 380s - los

In [19]:
final_loss, final_accuracy = model.evaluate(test_generator)
print(f"Final test loss: {final_loss}, Final test accuracy: {final_accuracy}")

158/158 [==============================] - 64s 405ms/step - loss: 0.2901 - accuracy: 0.8989
Final test loss: 0.2900563180446625, Final test accuracy: 0.8989298343658447


In [20]:
# Save the entire model
model.save('EfficientNetB3_BC.h5')
print("Saved model to disk.")

Saved model to disk.


In [6]:
def prepare_single_image(test_image_path, reference_image_path):
    test_img = cv2.imread(test_image_path, cv2.IMREAD_COLOR)
    test_img_resized = cv2.resize(test_img, (224, 224)).astype(np.float32) / 255.0

    reference_img = cv2.imread(reference_image_path, cv2.IMREAD_COLOR)
    reference_img_resized = cv2.resize(reference_img, (224, 224)).astype(np.float32) / 255.0

    difference_img = calculate_difference_image(test_img_resized, reference_img_resized)
    difference_img = np.expand_dims(difference_img, axis=0)
    return difference_img

# Load the saved model
loaded_model = load_model('EfficientNetB3_BC.h5')

test_image_path = 'D:\SASTRA Files\Sem-6\MINI PROJECT\Digital Image Forgery Detection\CASIA2 Dataset\Au\Au_ani_00001.jpg'
reference_image_path = 'D:\SASTRA Files\Sem-6\MINI PROJECT\Digital Image Forgery Detection\CASIA2 Dataset\Tp\Tp_D_CRN_S_B_nat10153_ani00001_12136.jpg'

difference_image = prepare_single_image(test_image_path, reference_image_path)
prediction = loaded_model.predict(difference_image)

print(f"Prediction (0 for Authentic, 1 for Tampered): {prediction[0]}")
if prediction[0] < 0.5:
    print("The image is likely tampered.")
else:
    print("The image is likely authentic.")



1/1 [==============================] - 3s 3s/step
Prediction (0 for Authentic, 1 for Tampered): [0.34223068]
The image is likely tampered.


In [13]:
# Load the saved model
loaded_model = load_model('EfficientNetB3_BC.h5')

test_image_path = 'D:\SASTRA Files\Sem-6\MINI PROJECT\Digital Image Forgery Detection\CASIA2 Dataset\Au\Au_pla_00050.jpg'
reference_image_path = 'D:\SASTRA Files\Sem-6\MINI PROJECT\Digital Image Forgery Detection\CASIA2 Dataset\Tp\Tp_D_NRN_S_N_ind00009_pla00050_10423.tif'

difference_image = prepare_single_image(test_image_path, reference_image_path)
prediction = loaded_model.predict(difference_image)

print(f"Prediction (0 for Authentic, 1 for Tampered): {prediction[0]}")
if prediction[0] < 0.5:
    print("The image is likely tampered.")
else:
    print("The image is likely authentic.")

1/1 [==============================] - 6s 6s/step
Prediction (0 for Authentic, 1 for Tampered): [0.02129186]
The image is likely tampered.


In [14]:
# Load the saved model
loaded_model = load_model('EfficientNetB3_BC.h5')

test_image_path = 'D:\SASTRA Files\Sem-6\MINI PROJECT\Digital Image Forgery Detection\CASIA2 Dataset\Au\Au_nat_00003.jpg'
reference_image_path = 'D:\SASTRA Files\Sem-6\MINI PROJECT\Digital Image Forgery Detection\CASIA2 Dataset\Au\Au_nat_00003.jpg'

difference_image = prepare_single_image(test_image_path, reference_image_path)
prediction = loaded_model.predict(difference_image)

print(f"Prediction (0 for Authentic, 1 for Tampered): {prediction[0]}")
if prediction[0] < 0.5:
    print("The image is likely tampered.")
else:
    print("The image is likely authentic.")

1/1 [==============================] - 7s 7s/step
Prediction (0 for Authentic, 1 for Tampered): [0.8264529]
The image is likely authentic.


In [15]:
# Load the saved model
loaded_model = load_model('EfficientNetB3_BC.h5')

test_image_path = 'D:\SASTRA Files\Sem-6\MINI PROJECT\Digital Image Forgery Detection\CASIA2 Dataset\Au\Au_txt_00065.jpg'
reference_image_path = 'D:\SASTRA Files\Sem-6\MINI PROJECT\Digital Image Forgery Detection\CASIA2 Dataset\Tp\Tp_D_NND_M_B_sec00077_txt00065_10388.tif'

difference_image = prepare_single_image(test_image_path, reference_image_path)
prediction = loaded_model.predict(difference_image)

print(f"Prediction (0 for Authentic, 1 for Tampered): {prediction[0]}")
if prediction[0] < 0.5:
    print("The image is likely tampered.")
else:
    print("The image is likely authentic.")

1/1 [==============================] - 6s 6s/step
Prediction (0 for Authentic, 1 for Tampered): [0.00537661]
The image is likely tampered.


In [16]:
# Load the saved model
loaded_model = load_model('EfficientNetB3_BC.h5')

test_image_path = 'D:\SASTRA Files\Sem-6\MINI PROJECT\Digital Image Forgery Detection\CASIA2 Dataset\Au\Au_nat_10124.jpg'
reference_image_path = 'D:\SASTRA Files\Sem-6\MINI PROJECT\Digital Image Forgery Detection\CASIA2 Dataset\Tp\Tp_D_NRN_M_N_cha10123_nat10124_12192.jpg'

difference_image = prepare_single_image(test_image_path, reference_image_path)
prediction = loaded_model.predict(difference_image)

print(f"Prediction (0 for Authentic, 1 for Tampered): {prediction[0]}")
if prediction[0] < 0.5:
    print("The image is likely tampered.")
else:
    print("The image is likely authentic.")

1/1 [==============================] - 6s 6s/step
Prediction (0 for Authentic, 1 for Tampered): [0.15594395]
The image is likely tampered.


In [17]:
# Load the saved model
loaded_model = load_model('EfficientNetB3_BC.h5')

test_image_path = 'D:\SASTRA Files\Sem-6\MINI PROJECT\Digital Image Forgery Detection\CASIA2 Dataset\Au\Au_ani_00032.jpg'
reference_image_path = 'D:\SASTRA Files\Sem-6\MINI PROJECT\Digital Image Forgery Detection\CASIA2 Dataset\Tp\Tp_D_NRN_S_N_ani00033_ani00032_10233.tif'

difference_image = prepare_single_image(test_image_path, reference_image_path)
prediction = loaded_model.predict(difference_image)

print(f"Prediction (0 for Authentic, 1 for Tampered): {prediction[0]}")
if prediction[0] < 0.5:
    print("The image is likely tampered.")
else:
    print("The image is likely authentic.")

1/1 [==============================] - 6s 6s/step
Prediction (0 for Authentic, 1 for Tampered): [0.20437609]
The image is likely tampered.


In [18]:
# Load the saved model
loaded_model = load_model('EfficientNetB3_BC.h5')

test_image_path = 'D:\SASTRA Files\Sem-6\MINI PROJECT\Digital Image Forgery Detection\CASIA2 Dataset\Au\Au_ani_00070.jpg'
reference_image_path = 'D:\SASTRA Files\Sem-6\MINI PROJECT\Digital Image Forgery Detection\CASIA2 Dataset\Au\Au_ani_00070.jpg'

difference_image = prepare_single_image(test_image_path, reference_image_path)
prediction = loaded_model.predict(difference_image)

print(f"Prediction (0 for Authentic, 1 for Tampered): {prediction[0]}")
if prediction[0] < 0.5:
    print("The image is likely tampered.")
else:
    print("The image is likely authentic.")

1/1 [==============================] - 6s 6s/step
Prediction (0 for Authentic, 1 for Tampered): [0.8264529]
The image is likely authentic.
